In [2]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
tf.reset_default_graph()

learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

# 기존의 신경망과 다르게 n_step으로 차원을 추가했다.
# 순서가 있는 데이터를 한 번에 입력받을 개수 와 총 몇단계로 이뤄진 데이터를 받을지를 설정해야 하기 때문.
# n_step은 행의 개수를 나타내므로 단계를 뜻하고, n_input은 열의 개수를 뜻하고, 한 단계의 데이터수를 나타냄.
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# 셀 생성
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

# 월래는 한 단계를 학습한 뒤 상태를 저장하고, 그 상태를 다음 단게의 입력 상태로 하여 다시 학습하는 단계를 반복시켜줘야하지만,
# TensorFlow에서 간단하게 구현이 가능하다
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# RNN에서 나오는 출력값을 가지고 최종출력값을 만드는 코드
# 그런데, RNN에서 나오는 출력값은 각 단계가 포함된 한 차원 높은 형태이다.
# 하지만 결과값을 얻기위해 행렬곱을 수행해야 하는데, 3차원행렬과 2차원행렬간의 곱은 할 수 없다.
# 그래서 n_step의 차원을 제거하여 마지막 단계의 결괏값만 취한다.

# tf.transpose() 함수를 이용해 n_step과 batch_size 차원의 순서를 바꾸고 n_step 차원을 제거함.
outputs = tf.transpose(outputs, [1, 0, 2])

# 마지막 단계의 결값만 취함.
outputs = outputs[-1]

# 최종결괏값 생성
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#########
#학습시작
#########
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Before outputs : \n', outputs)
print('Before states : \n', states)

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))
    

print('After outputs : \n', outputs)
print('After states : \n', states)    
print('최적화 완료!')

##########
#결과 확인
##########

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy,
                       feed_dict={X: test_xs, Y: test_ys}))



Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Before outputs : 
 Tensor("strided_slice:0", shape=(?, 128), dtype=float32)
Before states : 
 Tensor("rnn/while/Exit_2:0", shape=(?, 128), dtype=float32)
Epoch: 0001 Avg. cost =  0.535
Epoch: 0002 Avg. cost =  0.223
Epoch: 0003 Avg. cost =  0.177
Epoch: 0004 Avg. cost =  0.149
Epoch: 0005 Avg. cost =  0.133
Epoch: 0006 Avg. cost =  0.121
Epoch: 0007 Avg. cost =  0.116
Epoch: 0008 Avg. cost =  0.107
Epoch: 0009 Avg. cost =  0.101
Epoch: 0010 Avg. cost =  0.096
Epoch: 0011 Avg. cost =  0.099
Epoch: 0012 Avg. cost =  0.095
Epoch: 0013 Avg. cost =  0.091
Epoch: 0014 Avg. cost =  0.082
Epoch: 0015 Avg. cost =  0.085
Epoch: 0016 Avg. cost =  0.081
Epoch: 0017 Avg. cost =  0.079
Epoch: 0018 Avg. cost =  0.078
Epoch: 0019 Avg. cost =  0.076
Epoch: 0020 Avg. cost =  0.071
Epoch: 0021 Avg. cost = 